In [4]:
import pandas as pd
import sqlite3
import os

# create the sqlite DB
db = sqlite3.connect("hubway.db")

# read CSV into dataframe
df1 = pd.read_csv("Mobile_activity_3months_scrambled.csv", header=None)

# convert the pandas dataframe to a sqlite table
df1.to_sql("stage_expenses_import", db, if_exists="replace")

def run_query(query):
    return pd.read_sql_query(query,db)

os.getcwd()

C:\Users\kvinc\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'C:\\Users\\kvinc'

In [5]:
import pandas as pd
import sqlite3
import os

# create the sqlite DB
db = sqlite3.connect("hubway.db")

# read CSV into dataframe
df1 = pd.read_csv("Mobile_activity_3months_scrambled.csv", header=None)

# convert the pandas dataframe to a sqlite table


def run_query(query):
    return pd.read_sql_query(query,db)

os.getcwd()

'C:\\Users\\kvinc'

In [6]:
df1

,0,1,2,3
0,lat,lon,timestep,count
1,25.696395,-80.297496,1,1
2,25.699544,-80.297055,1,1
3,25.698612,-80.292015,1,1
4,25.939942,-80.341607,1,1
...,...,...,...,...
512322,49.0464,-122.781,3,1
512323,49.0458,-122.282,3,1
512324,49.052,-122.223,3,1
512325,49.0471,-122.365,3,1


In [9]:
query = """
SELECT * 
FROM sqlite_master
ORDER BY name;
"""
run_query(query)

,type,name,tbl_name,rootpage,sql
0,table,expenses,expenses,2,"CREATE TABLE ""expenses"" (\n""index"" INTEGER,\n ..."
1,index,ix_expenses_index,expenses,30,"CREATE INDEX ""ix_expenses_index""ON ""expenses"" ..."
2,index,ix_stage_expenses_import_index,stage_expenses_import,92,"CREATE INDEX ""ix_stage_expenses_import_index""O..."
3,table,stage_expenses_import,stage_expenses_import,46,"CREATE TABLE ""stage_expenses_import"" (\n""index..."


In [10]:
query = """
SELECT * 
FROM stage_expenses_import;
"""
run_query(query)

,index,0,1,2,3
0,0,lat,lon,timestep,count
1,1,25.696395,-80.297496,1,1
2,2,25.699544,-80.297055,1,1
3,3,25.698612,-80.292015,1,1
4,4,25.939942,-80.341607,1,1
...,...,...,...,...,...
512322,512322,49.046428,-122.780712,3,1
512323,512323,49.045792,-122.281553,3,1
512324,512324,49.052005,-122.2226,3,1
512325,512325,49.047132,-122.364595,3,1


In [11]:

# get information about my staging table 
query_table_help = """
SELECT name , *FROM PRAGMA_TABLE_INFO('stage_expenses_import');
"""
run_query(query_table_help)

,name,cid,name,type,notnull,dflt_value,pk
0,index,0,index,INTEGER,0,None,0
1,0,1,0,TEXT,0,None,0
2,1,2,1,TEXT,0,None,0
3,2,3,2,TEXT,0,None,0
4,3,4,3,TEXT,0,None,0


In [12]:

# example query showing date transform evolution & several other transforms 

query_format_columns = """
SELECT 
  "0" as "raw value"
 ,(trim("0")) as "TrimSpaces"
 ,substr((trim("0")) ,0,11) as "unformatted date substr"
 ,substr((trim("0")) ,7,4)||'-'||substr((trim("0")) ,1,2)||'-'||substr((trim("0")) ,4,2) as " formatted as ISO8601 str"
 ,datetime(substr((trim("0")) ,7,4)||'-'||substr((trim("0")) ,1,2)||'-'||substr((trim("0")) ,4,2)) as "ISO8601 as datetime"
 ,length(trim("2"))-1 as "column length-1" /* we get start position for the state substring below */
 ,substr("2" ,(length(trim("2"))-1) ,2) as "State" /* we dynamically substring the state from the last 2 characters */
 ,case (substr("2" ,(length(trim("2"))-1) ,2)) when "RK" then "NY" else (substr("2" ,(length(trim("2"))-1) ,2)) end as "Bad DEV Hard-Coded State"
 ,"6" as "Category"
 ,"8" as "Amount as real/float"
 ,"11" as "Vendor"
 ,cast(substr("13" ,1,5) as text) as "zc as txt"
 
from stage_expenses_import;
"""

query_distinct_card = """
SELECT 
  distinct "6" 
 
from stage_expenses_import;
"""

run_query(query_format_columns)
#run_query(query_distinct_card)

,raw value,TrimSpaces,unformatted date substr,formatted as ISO8601 str,ISO8601 as datetime,column length-1,State,Bad DEV Hard-Coded State,Category,Amount as real/float,Vendor,zc as txt
0,lat,lat,lat,-la-,None,7,ep,ep,6,8,11,13
1,25.696395,25.696395,25.696395,395-25-69,None,0,1,1,6,8,11,13
2,25.699544,25.699544,25.699544,544-25-69,None,0,1,1,6,8,11,13
3,25.698612,25.698612,25.698612,612-25-69,None,0,1,1,6,8,11,13
4,25.939942,25.939942,25.939942,942-25-93,None,0,1,1,6,8,11,13
...,...,...,...,...,...,...,...,...,...,...,...,...
512322,49.046428,49.046428,49.046428,428-49-04,None,0,3,3,6,8,11,13
512323,49.045792,49.045792,49.045792,792-49-04,None,0,3,3,6,8,11,13
512324,49.052005,49.052005,49.052005,005-49-05,None,0,3,3,6,8,11,13
512325,49.047132,49.047132,49.047132,132-49-04,None,0,3,3,6,8,11,13


In [13]:

# the query that populates stage_expenses_cleaned

query_transform_into_new_table = """
select 
  substr((trim("0")) ,7,4)||'-'||substr((trim("0")) ,1,2)||'-'||substr((trim("0")) ,4,2) as "Tran_Date"
 ,"11" as "Vendor_Name"
 ,case (substr("2" ,(length(trim("2"))-1) ,2)) when "RK" then "NY" else (substr("2" ,(length(trim("2"))-1) ,2)) end as "Vendor_State"
 ,cast(substr("13" ,1,5) as text) as "Vendor_Zip"
 ,"6" as "Vendor_Category Raw"
 ,case "11" when "AMAZON WEB SERVICES" then "Business Services" else "6" end as "Vendor_Category corrected"
 ,"8" as "Vendor_Amount"
from stage_expenses_import limit 5;
"""

query_distinct_card = """
select 
  distinct "6" 
 
from stage_expenses_import;
"""

run_query(query_transform_into_new_table)
#run_query(query_distinct_card)

,Tran_Date,Vendor_Name,Vendor_State,Vendor_Zip,Vendor_Category Raw,Vendor_Category corrected,Vendor_Amount
0,-la-,11,ep,13,6,6,8
1,395-25-69,11,1,13,6,6,8
2,544-25-69,11,1,13,6,6,8
3,612-25-69,11,1,13,6,6,8
4,942-25-93,11,1,13,6,6,8


In [14]:

# drop old table if exists 
c = db.cursor()

Drop_TableName = "stage_expenses_cleaned"

Drop_SQL = f"""
drop table if exists {Drop_TableName};
"""

c.execute(Drop_SQL)

c.close()

query_stage_expenses_cleaned = """
select 
*
from stage_expenses_cleaned;
"""

#run_query(query_stage_expenses_cleaned)
#run_query(query_distinct_card)

In [15]:

# create new table if not exists 
c = db.cursor()

Create_TableName = "stage_expenses_cleaned"

Create_SQL = f"""
create table if not exists {Create_TableName} as
select 
  substr((trim("0")) ,7,4)||'-'||substr((trim("0")) ,1,2)||'-'||substr((trim("0")) ,4,2) as "Tran_Date"
 ,"11" as "Vendor_Name"
 ,cast((case (substr("2" ,(length(trim("2"))-1) ,2)) when "RK" then "NY" else (substr("2" ,(length(trim("2"))-1) ,2)) end) as TEXT) as "Vendor_State"
 ,cast(substr("13" ,1,5) as text) as "Vendor_Zip"
 ,case "11" when "AMAZON WEB SERVICES" then "Business Services" else "6" end as "Vendor_Category"
 ,"8" as "Vendor_Amount"
from stage_expenses_import;
"""

c.execute(Create_SQL)

c.close()

stage_expenses_cleaned = """
select 
*
from stage_expenses_cleaned
order by Vendor_Amount desc;
"""

run_query(stage_expenses_cleaned)
#run_query(query_distinct_card)

,Tran_Date,Vendor_Name,Vendor_State,Vendor_Zip,Vendor_Category,Vendor_Amount
0,-la-,11,ep,13,6,8
1,395-25-69,11,1,13,6,8
2,544-25-69,11,1,13,6,8
3,612-25-69,11,1,13,6,8
4,942-25-93,11,1,13,6,8
...,...,...,...,...,...,...
512322,428-49-04,11,3,13,6,8
512323,792-49-04,11,3,13,6,8
512324,005-49-05,11,3,13,6,8
512325,132-49-04,11,3,13,6,8
